In [1]:
import openstudio

In [2]:
#房间的地板边界坐标点，按照顺时针排列。在EnergyPlus中面的坐标遵守右手准则，即四指按照坐标方向，大拇指应指向房间外。
points = [[0,0],
          [0,3],
          [3,3],
          [3,0]]

elevation = 0 #房间的高度，这里只有一个房间故为0.如果不止一层，就会大于0
floor_to_floor_height = 3 #楼层的高度

model = openstudio.model.Model() #生成存放建筑能耗模型的容器，也就是调用Model类

In [3]:
polygon = openstudio.Point3dVector() #openstudio有自己的数据格式，这里生成坐标点的列表

#将坐标点遍历，放到列表中
for point in points:
    x_coordinates = point[0]
    y_coordinates = point[1]
    openstudio_Point3d = openstudio.Point3d(x_coordinates,y_coordinates,elevation) 
    polygon.append(openstudio_Point3d)

In [4]:
story = openstudio.model.BuildingStory(model) #生成楼层的对象

space = openstudio.model.Space.fromFloorPrint(polygon, floor_to_floor_height, model).get() #将坐标放到openstudio自己的坐标格式
space.setBuildingStory(story) #将房间放到指定的楼层里

True

In [6]:
wwr = 0.5 #外墙窗前比
offset = 1 #窗户离地板高度
heightOffsetFromFloor = True

for s in model.getSurfaces(): #遍历模型中所有的面
    if s.outsideBoundaryCondition().lower() != "outdoors": #如果为外墙，则设置窗户
        continue

    new_window = s.setWindowToWallRatio(wwr, offset, heightOffsetFromFloor)

    if new_window.is_initialized(): #改名字
        new_window.get().setName(f"{s.nameString()} Window")

In [8]:
model.save(openstudio.path(r"shoebox_model.osm"))

True